In [5]:
import sinergym
from sinergym.utils.callbacks import LoggerEvalCallback
from sinergym.utils.rewards import *
from sinergym.utils.wrappers import LoggerWrapper
from datetime import datetime
import gym
import numpy as np
import gpytorch
import pandas as pd


environment = "Eplus-demo-v1"
episodes = 1
experiment_date = datetime.today().strftime('%Y-%m-%d %H:%M')

# register run name
name = F"DQN-{environment}-episodes_{episodes}({experiment_date})"


extra_conf={
    'timesteps_per_hour':4,
    'runperiod':(1,1,1991,2,1,1993),
}
env = gym.make(environment, 
                reward=LinearReward, 
                config_params=extra_conf)


env = LoggerWrapper(env)

data = pd.DataFrame()

print(env.action_space)

for i in range(100):
    print(env.action_space.sample())

print(env.action_mapping)

# obs = env.reset()
# for timestep in range(1, 100000):
#     action = env.action_space.sample()
#     obs, reward, done, info = env.step(action)
#     # print(obs)
#     if done:
#         env.reset()


env.close()

[2023-04-09 01:31:35,353] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-04-09 01:31:35,353] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-04-09 01:31:35,353] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-04-09 01:31:35,353] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-04-09 01:31:35,353] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-04-09 01:31:35,356] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-04-09 01:31:35,356] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-04-09 01:31:35,356] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Up

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
class MyGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)

        # Mean Function
        self.mean_module = gpytorch.means.ZeroMean()

        # Kernel Function
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean = self.mean_module(x)
        covar = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean, covar)

# train_x = ...; train_y = ...
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = MyGP(train_x, train_y, likelihood)


# Put model in train mode
model.train()
likelihood.train()

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# Define loss function the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# training step
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss
    loss = -mll(output, train_y)
    #  backprop gradients
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()